In [56]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
import logging

In [58]:
import lmsquery
import requests
from epdlib.Screen import Update
from copy import copy

In [59]:
import QueryLMS

In [60]:
import sys
from pathlib import Path

In [61]:
# # two different import modes for development or distribution
# try:
#     # import from other modules above this level
#     from .layout import layout
# except ImportError:
#     # development in jupyter notebook
#     from layout import layout

In [62]:
try:
    from . import layout
    from . import constants
except ImportError:
    import layout
    import constants

In [63]:
logger = logging.getLogger(__name__)

In [64]:
def update_function(self):
    '''update_function for Plugin() object to read data from a 
    Logitech Media Server and show now-playing information for a single player
    multiple players can be tracked by adding multiple plugins
    
    Requirements:
        self.config(`dict`): {
            'player_name': 'LMS Player Name',   # name of player to track
            'idle_timeout': 10,                 # timeout for showing 'pause' screen 
        }
        self.cache(`CacheFiles` object)
            
    Args:
        self(namespace): namespace from plugin object
    %U'''
    def build_lms():
        logging.debug(f'building LMS Query object for player: {player_name}')
#         self.my_lms = lmsquery.LMSQuery(player_name=player_name)
        self.my_lms = QueryLMS.QueryLMS(player_name=player_name)
    
    logging.debug(f'update_function for plugin {self.name}, version {constants.version}')
    now_playing = None
    # make a shallow copy to make updates possible without impacting origonal obj.
    data = copy(constants.data)
    is_updated = False
    priority = 2**15    
 
    
    failure = (is_updated, data, priority)
    
    player_name = self.config['player_name']
    
    if not hasattr(self, 'play_state'):
        self.play_state = 'None'
    
    # add the idle timer on first run
    if not hasattr(self, 'idle_timer'):
        logging.debug(f'adding idle_timer of class `Update()`')
        self.idle_timer = Update()
    
    # check if LMS Query object is initiated
    if not hasattr(self, 'my_lms'):
        # add LMSQuery object to self
#         logging.debug(f'building LMS Query object for player: {player_name}')
#         self.my_lms = lmsquery.LMSQuery(player_name=player_name)
        build_lms()
    try:
        # fetch the now playing data for the player
        now_playing = self.my_lms.get_now_playing()
        # remove the time key to make comparisions now_playing data updates easier in the Plugin class
        if 'time' in now_playing:
            now_playing.pop('time')
    
    # this should cover most network related errors
    except requests.exceptions.ConnectionError as e:
        logging.error(f'network error finding player "{player_name}": {e}')
        logging.info(f'rebuilding LMS Query object for {player_name}')
        build_lms()
        return failure
    # if no data is returned, pulling 'time' key throws key error
    except KeyError as e:
        logging.warning(f'error getting now plyaing information for "{player_name}": KeyError {e}')
        logging.warning('this error is typical of newly added player or player that has no "now playing" data')
        return failure
    # QueryLMS throws ValueError if player_id is not set 
    except ValueError as e:
        logging.warning(f'could not get now playing information for "{player_name}": ValueError {e}')
        logging.warning(f'check player_name in config file. Is "{player_name}" connected to the LMS server?')
        return failure
    
    
    
    # process the now_playing state and set priority, update and data
    if now_playing:
        data = now_playing
        try:
            data['coverart'] = self.cache.cache_file(now_playing['artwork_url'], 
                                                     now_playing['album_id'])
        except KeyError as e:
            logging.warning(f'failed to cache file -- now_playing data did not contain complete data: {e}')
    logging.debug(f'now_playing: {now_playing["mode"]}')
    if now_playing['mode'] == 'play':
        if self.data == data:
            priority = self.max_priority
        else:
            priority = self.max_priority - 1
        self.play_state = 'play'
        is_updated = True
        
    elif now_playing['mode'] == 'pause':
        # moving from play to pause, decrease priority and refresh idle_timer
        if self.play_state == 'play':
            self.idle_timer.update()
            priority = self.max_priority + 1
            self.play_state = 'pause'
        
        # if the idle timer has expired, decrease priority
        if self.idle_timer.last_updated > self.config['idle_timeout']:
            priority = self.max_priority + 3
            self.play_state = 'pause'
        else:
            priority = self.max_priority + 1

        is_updated = True
    
    else: 
        self.play_state = now_playing['mode'] 
        priority = 2**15
        is_updated = False
    logging.info(f'priority set to: {priority}')
    return (is_updated, data, priority)

In [67]:
# from SelfDummy import SelfDummy
# from CacheFiles import CacheFiles


# logger.root.setLevel('DEBUG')
# logging.debug('foo')

# self = SelfDummy()
# self.max_priority = 0
# self.config = {'player_name': 'slimpi',
#                'idle_timeout': 5}
# self.cache = CacheFiles()

DEBUG:root:foo


In [73]:
# u, d, p = update_function(self)
# if u != self.data:
#     self.data = d
# print(f'idle timer: {self.idle_timer.last_updated}, idle_timeout {self.config["idle_timeout"]}')
# print(p)
# print(d)


DEBUG:root:update_function for plugin None, version 0.1.0
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 192.168.178.9:9000
DEBUG:urllib3.connectionpool:http://192.168.178.9:9000 "POST /jsonrpc.js HTTP/1.1" 200 295
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 192.168.178.9:9000
DEBUG:urllib3.connectionpool:http://192.168.178.9:9000 "POST /jsonrpc.js HTTP/1.1" 200 344
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 192.168.178.9:9000
DEBUG:urllib3.connectionpool:http://192.168.178.9:9000 "POST /jsonrpc.js HTTP/1.1" 200 258
DEBUG:root:caching file from url http://192.168.178.9:9000/music/c9d646ff/cover.jpg to /tmp/rjewg6sv/2064
DEBUG:root:file previously cached
DEBUG:root:now_playing: stop
INFO:root:priority set to: 32768


idle timer: 14.515546538983472, idle_timeout 5
32768
{'mode': 'stop', 'id': 17001, 'title': 'We Belong Together', 'artist': 'Vampire Weekend feat. Danielle Haim', 'coverid': 'c9d646ff', 'duration': 190.733, 'album_id': '2064', 'genre': 'No Genre', 'album': 'Father of the Bride', 'artwork_url': 'http://192.168.178.9:9000/music/c9d646ff/cover.jpg', 'coverart': PosixPath('/tmp/rjewg6sv/2064')}


In [34]:
def scan_servers(*args, **kwargs):
    """scan for and list all available LMS Servers and players on the local network
    
    usage:
        lms_client.scan_servers
        
    Args:
        None
    Returns:
        None
    %U"""
    print(f'Scanning for available LMS Server and players')
    servers = QueryLMS.QueryLMS().scan_lms()
    if not servers:
        print('Error: no LMS servers were found on the network. Is there one running?')
        do_exit(1)
    print('servers found:')
    print(servers)
    players = QueryLMS.QueryLMS().get_players()
    # print selected keys for each player
    keys = ['name', 'playerid', 'modelname']
    for p in players:
        print('players found:')
        try:
            for key in keys:
                print(f'{key}: {p[key]}')
            print('\n')
        except KeyError as e:
            pass 

In [ ]:
!jupyter-nbconvert --to python --template python_clean lms_client.ipynb


In [ ]:
!rm ./SelfDummy.py
!rm ./CacheFiles.py

In [ ]:
!ln -s ../../library/SelfDummy.py 
!ln -s ../../library/CacheFiles.py

## Crash
```
10:43:30 paperpi:update_plugins:365:INFO - ***************
10:43:30 paperpi:update_plugins:366:INFO - updating 5 plugins
10:43:30 word_clock:update_function:81:INFO - update_function for Plugin: Word Clock
10:43:30 Layout:update_contents:382:INFO - updating blocks
10:43:31 paperpi:update_plugins:369:INFO - update: [Plugin: Word Clock]-p: 2
10:43:31 paperpi:update_plugins:369:INFO - update: [Plugin: Decimal Binary Clock]-p: 2
10:43:32 lms_client:update_function:119:ERROR - could not find player "MacPlay": ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
10:43:32 lms_client:update_function:120:INFO - rebuilding LMS Query object for MacPlay
10:43:37 lmsquery:scanLMS:181:WARNING - server search timed out after 5 seconds with no results
10:43:37 lmsquery:lms_servers:72:WARNING - no servers found on local network
Traceback (most recent call last):
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/urllib3/connectionpool.py", line 384, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3.7/http/client.py", line 1336, in getresponse
    response.begin()
  File "/usr/lib/python3.7/http/client.py", line 306, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.7/http/client.py", line 275, in _read_status
    raise RemoteDisconnected("Remote end closed connection without"
http.client.RemoteDisconnected: Remote end closed connection without response

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/requests/adapters.py", line 449, in send
    timeout=timeout
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/urllib3/connectionpool.py", line 638, in urlopen
    _stacktrace=sys.exc_info()[2])
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/urllib3/util/retry.py", line 368, in increment
    raise six.reraise(type(error), error, _stacktrace)
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/urllib3/packages/six.py", line 685, in reraise
    raise value.with_traceback(tb)
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/urllib3/connectionpool.py", line 384, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3.7/http/client.py", line 1336, in getresponse
    response.begin()
  File "/usr/lib/python3.7/http/client.py", line 306, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.7/http/client.py", line 275, in _read_status
    raise RemoteDisconnected("Remote end closed connection without"
urllib3.exceptions.ProtocolError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pi/src/epd_display/paperpi/plugins/lms_client/lms_client.py", line 113, in update_function
    now_playing = self.my_lms.now_playing()
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/lmsquery/lmsquery.py", line 290, in now_playing
    status = self.query(player_id, 'status')
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/lmsquery/lmsquery.py", line 196, in query
    r = requests.post(self.server_url, params)
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/requests/api.py", line 116, in post
    return request('post', url, data=data, json=json, **kwargs)
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/requests/api.py", line 60, in request
    return session.request(method=method, url=url, **kwargs)
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/requests/sessions.py", line 533, in request
    resp = self.send(prep, **send_kwargs)
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/requests/sessions.py", line 646, in send
    r = adapter.send(request, **kwargs)
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/requests/adapters.py", line 498, in send
    raise ConnectionError(err, request=request)
requests.exceptions.ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/urllib3/connection.py", line 159, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/urllib3/util/connection.py", line 57, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/usr/lib/python3.7/socket.py", line 748, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/urllib3/connectionpool.py", line 354, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/usr/lib/python3.7/http/client.py", line 1244, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/usr/lib/python3.7/http/client.py", line 1290, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/usr/lib/python3.7/http/client.py", line 1239, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/usr/lib/python3.7/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/usr/lib/python3.7/http/client.py", line 966, in send
    self.connect()
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/urllib3/connection.py", line 181, in connect
    conn = self._new_conn()
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/urllib3/connection.py", line 168, in _new_conn
    self, "Failed to establish a new connection: %s" % e)
urllib3.exceptions.NewConnectionError: <urllib3.connection.HTTPConnection object at 0xb54f28f0>: Failed to establish a new connection: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/requests/adapters.py", line 449, in send
    timeout=timeout
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/urllib3/connectionpool.py", line 638, in urlopen
    _stacktrace=sys.exc_info()[2])
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/urllib3/util/retry.py", line 399, in increment
    raise MaxRetryError(_pool, url, error or ResponseError(cause))
urllib3.exceptions.MaxRetryError: HTTPConnectionPool(host='none', port=9000): Max retries exceeded with url: /jsonrpc.js (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0xb54f28f0>: Failed to establish a new connection: [Errno -2] Name or service not known'))

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "paperpi.py", line 536, in <module>
    c = main()
  File "paperpi.py", line 520, in main
    update_loop(plugins, screen)
  File "paperpi.py", line 407, in update_loop
    priority_list = update_plugins()
  File "paperpi.py", line 368, in update_plugins
    plugin.update()
  File "/home/pi/src/epd_display/paperpi/library/Plugin.py", line 220, in update
    is_updated, data, priority = self.update_function(*args, **kwargs)
  File "/home/pi/src/epd_display/paperpi/plugins/lms_client/lms_client.py", line 121, in update_function
    build_lms()
  File "/home/pi/src/epd_display/paperpi/plugins/lms_client/lms_client.py", line 83, in build_lms
    self.my_lms = lmsquery.LMSQuery(player_name=player_name)
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/lmsquery/lmsquery.py", line 58, in __init__
    self.player_name = player_name
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/lmsquery/lmsquery.py", line 95, in player_name
    for p in self.get_players():
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/lmsquery/lmsquery.py", line 221, in get_players
    players = self.get_server_status()
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/lmsquery/lmsquery.py", line 206, in get_server_status
    return self.query("", "serverstatus", 0, 99)
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/lmsquery/lmsquery.py", line 196, in query
    r = requests.post(self.server_url, params)
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/requests/api.py", line 116, in post
    return request('post', url, data=data, json=json, **kwargs)
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/requests/api.py", line 60, in request
    return session.request(method=method, url=url, **kwargs)
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/requests/sessions.py", line 533, in request
    resp = self.send(prep, **send_kwargs)
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/requests/sessions.py", line 646, in send
    r = adapter.send(request, **kwargs)
  File "/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/requests/adapters.py", line 516, in send
    raise ConnectionError(e, request=request)
requests.exceptions.ConnectionError: HTTPConnectionPool(host='none', port=9000): Max retries exceeded with url: /jsonrpc.js (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0xb54f28f0>: Failed to establish a new connection: [Errno -2] Name or service not known'))
(epd_display) 11:17:30
```